## Tax Primer

In [1]:
import pandas as pd
import numpy as np
!pip install numpy_financial
import numpy_financial as npf
import plotly.graph_objects as go
import plotly.express as px
pd.options.display.float_format = '{:,.2f}'.format


[notice] A new release of pip available: 22.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
LIMITS = [0, 10275, 41775, 89075, 170050, 215950, 539900]
RATES  = [0.1, 0.12, 0.22, 0.24, 0.32, 0.35, 0.37]

cols = ['lower','upper','rate','bin_income','bin_tax']
df = pd.DataFrame(dtype=float, columns=cols, index=1+np.arange(len(LIMITS)))
df['lower'] = LIMITS
df['upper'] = LIMITS[1:] + [np.inf]
df['rate']  = RATES
df

,lower,upper,rate,bin_income,bin_tax
1,0,"10,275.00",0.10,NaN,NaN
2,10275,"41,775.00",0.12,NaN,NaN
3,41775,"89,075.00",0.22,NaN,NaN
4,89075,"170,050.00",0.24,NaN,NaN
5,170050,"215,950.00",0.32,NaN,NaN
6,215950,"539,900.00",0.35,NaN,NaN
7,539900,inf,0.37,NaN,NaN


In [3]:
INCOME = 100000
df['bin_income']= np.where(INCOME>df.lower, 
                    np.where(INCOME>df.upper, df.upper-df.lower, INCOME-df.lower),
                    0)
df

,lower,upper,rate,bin_income,bin_tax
1,0,"10,275.00",0.10,"10,275.00",NaN
2,10275,"41,775.00",0.12,"31,500.00",NaN
3,41775,"89,075.00",0.22,"47,300.00",NaN
4,89075,"170,050.00",0.24,"10,925.00",NaN
5,170050,"215,950.00",0.32,0.00,NaN
6,215950,"539,900.00",0.35,0.00,NaN
7,539900,inf,0.37,0.00,NaN


In [4]:
# Check that sum of df.bin_income equals taxable income
df.bin_income.sum()

100000.0

In [5]:
df['bin_tax'] = df.bin_income*df.rate
df

,lower,upper,rate,bin_income,bin_tax
1,0,"10,275.00",0.10,"10,275.00","1,027.50"
2,10275,"41,775.00",0.12,"31,500.00","3,780.00"
3,41775,"89,075.00",0.22,"47,300.00","10,406.00"
4,89075,"170,050.00",0.24,"10,925.00","2,622.00"
5,170050,"215,950.00",0.32,0.00,0.00
6,215950,"539,900.00",0.35,0.00,0.00
7,539900,inf,0.37,0.00,0.00


In [6]:
# Total tax
total_tax = df.bin_tax.sum()
print(f'Total tax: ${total_tax:,.2f}')

Total tax: $17,835.50


In [7]:
def tax_single_filer(income):
    LIMITS = [0, 10275, 41775, 89075, 170050, 215950, 539900]
    RATES  = [0.1, 0.12, 0.22, 0.24, 0.32, 0.35, 0.37]
    cols = ['lower','upper','rate','bin_income','bin_tax']
    df = pd.DataFrame(dtype=float, columns=cols, index=1+np.arange(len(LIMITS)))
    df['lower'] = LIMITS
    df['upper'] = LIMITS[1:] + [np.inf]
    df['rate']  = RATES
    df['bin_income']= np.where(income>df.lower, 
                    np.where(income>df.upper, df.upper-df.lower, income-df.lower),
                    0)
    df['bin_tax'] = df.bin_income*df.rate
    return df.bin_tax.sum()

In [8]:
incomes = np.arange(1000,1001000,1000)
taxes = [tax_single_filer(x) for x in incomes]

In [9]:
# Scatter plot of tax due
fig = go.Figure()
trace  = go.Scatter(x=incomes, y=taxes, mode="lines")
fig.add_trace(trace)

# Formatting
fig.update_xaxes(title='Income (after deductions)',tickformat=",.2f")
fig.update_yaxes(title='Tax',tickformat=",.2f")
fig.show()

In [10]:
# Scatter plot of average tax rate
fig = go.Figure()
trace  = go.Scatter(x=incomes, y=taxes/incomes, mode="lines")
fig.add_trace(trace)

# Formatting
fig.update_xaxes(title='Income (after deductions)',tickformat=",.0f")
fig.update_yaxes(title='Average Tax Rate',tickformat=",.2%")
fig.show()

### Example

- Consider a single investor earning $65,000 per year.  
- Assume they take the standard deduction of $12,590.
- Under the 2022 tax rates & brackets, what is the investor's tax bill if do not have any other deductions?

In [11]:
GROSS_INCOME = 80000
DEDUCTIBLE_SAVINGS = 0
STANDARD_DEDUCTION = 12590
tax1 = tax_single_filer(GROSS_INCOME - DEDUCTIBLE_SAVINGS - STANDARD_DEDUCTION)
tax1

10447.2

- Under the 2022 tax rates & brackets, what is the investor's tax bill if they have contributed $6,000 to a traditional IRA?

In [12]:
GROSS_INCOME = 80000
DEDUCTIBLE_SAVINGS = 6000
STANDARD_DEDUCTION = 12590
tax2 = tax_single_filer(GROSS_INCOME - DEDUCTIBLE_SAVINGS - STANDARD_DEDUCTION)
tax2

9127.2

In [13]:
print(RATES[2]*DEDUCTIBLE_SAVINGS)
print(tax1-tax2)

1320.0
1320.0
